This notebook is based on [this example from Francois Chollet](https://github.com/fchollet/keras/blob/master/examples/reuters_mlp.py). 

# Keras
Keras - библиотека с открытым кодом для разработки нейронный сетей. Ее автор Франсуа Шолле. Изначально задумывалась как надстройка над Theano, но теперь по умолчанию используется TensorFlow.

<img src="Images/Keras_Logo.jpg" style="width:200px;height:200px;">


# Установка
Установка Keras чрезвычайно проста, т.к. он является обычным питоновским пакетом

In [1]:
!pip install keras

Чтобы работать с Keras, у вас уже должен быть установлен хотя бы один из фреймворков — Theano или Tensorflow.
Бэкенды — это то, из-за чего Keras стал известен и популярен (помимо прочих достоинств, которые мы разберем ниже). Keras позволяет использовать в качестве бэкенда разные другие фреймворки. При этом написанный вами код будет исполняться независимо от используемого бэкенда. Начиналась разработка, как мы уже говорили, с Theano, но со временем добавился Tensorflow и CNTK.

In [2]:
!pip install tensorflow

# Практический пример
## Данные
Обучение любой модели в машинном обучении начинается с данных. Keras содержит внутри несколько обучающих датасетов, но они уже приведены в удобную для работы форму и не позволяют показать всю мощь Keras. Поэтому мы возьмем более сырой датасет. Это будет датасет 20 newsgroups — 20 тысяч новостных сообщений из групп Usenet (это такая система обмена почтой родом из 1990-х) примерно поровну распределенных по 20 категориям. Мы будем учить нашу сеть правильно распределять сообщения по этим новостным группам.

In [3]:
import numpy as np
import keras

/Users/dosya/Projects/lab_iis/jupyter/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [5]:
!pip install --upgrade httplib2

    100% |████████████████████████████████| 225kB 1.2MB/s ta 0:00:01    99% |███████████████████████████████▉| 215kB 1.1MB/s eta 0:00:01
  Running setup.py bdist_wheel for httplib2 ... done
  Stored in directory: /Users/dosya/Library/Caches/pip/wheels/1b/9c/9e/1f6fdb21dbb1fe6a99101d697f12cb8c1fa96c1587df69adba
Successfully built httplib2


In [6]:
from sklearn.datasets import fetch_20newsgroups
newsgroups_train = fetch_20newsgroups(subset='train')
newsgroups_test = fetch_20newsgroups(subset='test')

URLError: <urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed (_ssl.c:749)>

In [16]:
print(len(newsgroups_train["data"]))
print(len(newsgroups_test["data"]))

11314
7532


In [9]:
print(newsgroups_train["data"][0])

From: lerxst@wam.umd.edu (where's my thing)
Subject: WHAT car is this!?
Nntp-Posting-Host: rac3.wam.umd.edu
Organization: University of Maryland, College Park
Lines: 15

 I was wondering if anyone out there could enlighten me on this car I saw
the other day. It was a 2-door sports car, looked to be from the late 60s/
early 70s. It was called a Bricklin. The doors were really small. In addition,
the front bumper was separate from the rest of the body. This is 
all I know. If anyone can tellme a model name, engine specs, years
of production, where this car is made, history, or whatever info you
have on this funky looking car, please e-mail.

Thanks,
- IL
   ---- brought to you by your neighborhood Lerxst ----





1000


## Предпроцессинг
Keras содержит в себе инструменты для удобного препроцессинга текстов, картинок и временных рядов, иными словами, самых распространенных типов данных. В этом туториале мы работаем с текстами, поэтому нам нужно разбить их на токены и привести в матричную форму.

In [4]:
from keras.preprocessing.text import Tokenizer
print("Preparing the Tokenizer...")
max_words = 1000

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(newsgroups_train["data"])

Preparing the Tokenizer...


In [5]:
print('Vectorizing sequence data...')
x_train = tokenizer.texts_to_matrix(newsgroups_train["data"], mode='binary')
x_test = tokenizer.texts_to_matrix(newsgroups_test["data"], mode='binary')
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

Vectorizing sequence data...
x_train shape: (11314, 1000)
x_test shape: (7532, 1000)


Еще нам понадобится преобразовать метки классов к матричному виду для обучения с помощью кросс-энтропии. Для этого мы переведем номер класса в так называемый one-hot вектор, т.е. вектор, состоящий из нулей и одной единицы.

In [20]:
num_classes = np.max(newsgroups_train["target"]) + 1
print(num_classes, 'classes')

20 classes


In [21]:
print('Convert class vector to binary class matrix '
      '(for use with categorical_crossentropy)')
y_train = keras.utils.to_categorical(newsgroups_train["target"], num_classes)
y_test = keras.utils.to_categorical(newsgroups_test["target"], num_classes)
print('y_train shape:', y_train.shape)
print('y_test shape:', y_test.shape)

Convert class vector to binary class matrix (for use with categorical_crossentropy)
y_train shape: (11314, 20)
y_test shape: (7532, 20)


# Модель
Модель в Keras можно описать двумя основными способами:

## Sequential API

Первый — последовательное описание модели, например, вот так:

In [22]:
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Input
print('Building model sequentially 1...')
model = Sequential()
model.add(Dense(512, input_shape=(max_words,)))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

Building model sequentially 1...


или вот так:

In [ ]:
print('Building model sequentially 2...')
model = Sequential([
          Dense(512, input_shape=(max_words,)),
          Activation('relu'),
          Dropout(0.5),
          Dense(num_classes),
          Activation('softmax')
        ])

## Functional API

Некоторое время назад появилась возможность использовать функциональное API для создания модели — второй способ:

In [80]:
print('Building model functionally...')
a = Input(shape=(max_words,))
b = Dense(512)(a)
b = Activation('relu')(b)
b = Dropout(0.5)(b)
b = Dense(num_classes)(b)
b = Activation('softmax')(b)
model = Model(inputs=a, outputs=b)

Building model functionally...


Класс Model (и унаследованный от него Sequential) имеет удобный интерфейс, позволяющий посмотреть, какие слои входят в модель — model.layers, входы — model.inputs, и выходы — model.outputs.

Посмотреть количество параметров модели можно model.summary()


Также очень удобный метод отображения и сохранения модели в человеко-читаемом виде — model.to_yaml. Можно инстанциировать модели из такого описания.

In [ ]:
print(model.layers)

In [27]:
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 512)               512512    
_________________________________________________________________
activation_1 (Activation)    (None, 512)               0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 20)                10260     
_________________________________________________________________
activation_2 (Activation)    (None, 20)                0         
Total params: 522,772
Trainable params: 522,772
Non-trainable params: 0
_________________________________________________________________
None


In [28]:
from keras.models import model_from_yaml

yaml_string = model.to_yaml()
print(yaml_string)
model = model_from_yaml(yaml_string)

backend: tensorflow
class_name: Sequential
config:
- class_name: Dense
  config:
    activation: linear
    activity_regularizer: null
    batch_input_shape: !!python/tuple [null, 1000]
    bias_constraint: null
    bias_initializer:
      class_name: Zeros
      config: {}
    bias_regularizer: null
    dtype: float32
    kernel_constraint: null
    kernel_initializer:
      class_name: VarianceScaling
      config: {distribution: uniform, mode: fan_avg, scale: 1.0, seed: null}
    kernel_regularizer: null
    name: dense_1
    trainable: true
    units: 512
    use_bias: true
- class_name: Activation
  config: {activation: relu, name: activation_1, trainable: true}
- class_name: Dropout
  config: {name: dropout_1, noise_shape: null, rate: 0.5, seed: null, trainable: true}
- class_name: Dense
  config:
    activation: linear
    activity_regularizer: null
    bias_constraint: null
    bias_initializer:
      class_name: Zeros
      config: {}
    bias_regularizer: null
    kernel_cons

## Подготовка модели к работе
Что означают параметры функции compile? loss — это функция ошибки, в нашем случае — это перекрестная энтропия, именно для нее мы подготавливали наши метки в виде матриц; optimizer — используемый оптимизатор, здесь мог бы быть обычный стохастический градиентный спуск, но Adam показывает лучшую сходимость на этой задаче; metrics — метрики, по которым считается качество модели, в нашем случае — это точность (accuracy), то есть доля верно угаданных ответов.

In [29]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

Несмотря на то, что Keras содержит большинство популярных функций ошибки, для вашей задачи может потребоваться что-то уникальное. Чтобы сделать свой собственный loss, нужно немного: просто определить функцию, принимающую векторы правильных и предсказанных ответов и выдающую одно число на выход. Для тренировки сделаем свою функцию расчета перекрестной энтропии. Чтобы она чем-то отличалась, введем так называемый clipping — обрезание значений вектора сверху и снизу. Да, еще важное замечание: нестандартный loss может быть необходимо описывать в терминах нижележащего фреймворка, но мы можем обойтись средствами Keras.

In [30]:
from keras.objectives import categorical_crossentropy
from keras import backend as K

epsilon = 1.0e-9
def custom_objective(y_true, y_pred):
    '''Yet another crossentropy'''
    y_pred = K.clip(y_pred, epsilon, 1.0 - epsilon)
    y_pred /= K.sum(y_pred, axis=-1, keepdims=True)
    cce = categorical_crossentropy(y_pred, y_true)
    return cce

In [31]:
model.compile(loss=custom_objective,
              optimizer='adam',
              metrics=['accuracy'])

# Обучение и тестирование

Метод fit делает именно это. Он принимает на вход обучающую выборку вместе с метками — x_train и y_train, размером батча batch_size, который ограничивает количество примеров, подаваемых за раз, количеством эпох для обучения epochs (одна эпоха — это один раз полностью пройденная моделью обучающая выборка), а также тем, какую долю обучающей выборки отдать под валидацию — validation_split.

Возвращает этот метод history — это история ошибок на каждом шаге обучения.

In [32]:
batch_size = 32
epochs = 5
history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1)

Train on 10182 samples, validate on 1132 samples
Epoch 1/5
10182/10182 [==============================] - 5s 529us/step - loss: 10.6697 - acc: 0.3935 - val_loss: 7.4356 - val_acc: 0.5928
Epoch 2/5
10182/10182 [==============================] - 5s 480us/step - loss: 6.6719 - acc: 0.6243 - val_loss: 5.9738 - val_acc: 0.6661
Epoch 3/5
10182/10182 [==============================] - 4s 436us/step - loss: 5.2289 - acc: 0.7081 - val_loss: 5.5104 - val_acc: 0.6837
Epoch 4/5
10182/10182 [==============================] - 4s 438us/step - loss: 4.5327 - acc: 0.7426 - val_loss: 5.3232 - val_acc: 0.6890
Epoch 5/5
10182/10182 [==============================] - 5s 513us/step - loss: 4.1579 - acc: 0.7617 - val_loss: 5.2844 - val_acc: 0.6890


И наконец, тестирование. Метод evaluate получает на вход тестовую выборку вместе с метками для нее. Метрика была задана еще при подготовке к работе, так что больше ничего не нужно. (Но мы укажем еще размер батча).

In [ ]:
score = model.evaluate(x_test, y_test, batch_size=batch_size)

## Callbacks

Нужно также сказать несколько слов о такой важной особенности Keras, как колбеки. Через них реализовано много полезной функциональности. Например, если вы тренируете сеть в течение очень долгого времени, вам нужно понять, когда пора остановиться, если ошибка на вашем датасете перестала уменьшаться. По-английски описываемая функциональность называется "early stopping" ("ранняя остановка"). Посмотрим, как мы можем применить его при обучении нашей сети:

In [77]:
from keras.callbacks import EarlyStopping  
early_stopping=EarlyStopping(monitor='val_acc', patience=2, mode='max', min_delta=0.0001, verbose=1)  
print(early_stopping.monitor)

epochs = 15
history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_test, y_test),
                    callbacks=[early_stopping])

val_acc
Train on 11314 samples, validate on 7532 samples
Epoch 1/15
11314/11314 [==============================] - 7s 620us/step - loss: 0.4577 - acc: 0.9720 - val_loss: 5.5979 - val_acc: 0.6545
Epoch 2/15
11314/11314 [==============================] - 7s 613us/step - loss: 0.4526 - acc: 0.9723 - val_loss: 5.4680 - val_acc: 0.6634
Epoch 3/15
11314/11314 [==============================] - 7s 631us/step - loss: 0.4543 - acc: 0.9722 - val_loss: 5.5714 - val_acc: 0.6556
Epoch 4/15
11314/11314 [==============================] - 7s 619us/step - loss: 0.4417 - acc: 0.9731 - val_loss: 5.4472 - val_acc: 0.6624
Epoch 00004: early stopping


In [79]:
score = model.evaluate(x_test, y_test,
                       batch_size=batch_size, verbose=1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])

7532/7532 [==============================] - 1s 97us/step


Test score: 5.447168237024713
Test accuracy: 0.6623738714816781


# Заключение

Пришло время обсудить плюсы и минусы Keras. К очевидным плюсам можно отнести простоту создания моделей, которая выливается в высокую скорость прототипирования. В целом этот фреймворк становится все более и более популярным. Кажется, своей цели — простоты использования — Франсуа Шолле (François Chollet, автор Keras) добился. Более того, его инициатива не осталась незамеченной: буквально через несколько месяцев разработки компания Google пригласила его заниматься этим в команде, разрабатывающей Tensorflow. А также с версии Tensorflow 1.2 Keras был включен в состав TF (tf.keras).


Также надо сказать пару слов о недостатках. К сожалению, идея Keras о универсальности кода выполняется не всегда: Keras 2.0 поломал совместимость с первой версией, некоторые функции стали называться по-другому, некоторые переехали, в общем, история похожа на второй и третий python. Отличием является то, что в случае Keras была выбрана только вторая версия для развития. Также код Keras работает на Tensorflow пока медленнее, чем на Theano (хотя для нативного кода фреймворки, как минимум, сравнимы).


В целом, можно порекомендовать Keras к использованию, когда вам нужно быстро составить и протестировать сеть для решения конкретной задачи. Но если вам нужны какие-то сложные вещи, вроде нестандартного слоя или распараллеливания кода на несколько GPU, то лучше (а подчас просто неизбежно) использовать нижележащий фреймворк.